# 4. External Quantification Tools

In [1]:
import suspect
import numpy as np
from matplotlib import pyplot as plt
%matplotlib nbagg

In [2]:
data = suspect.io.load_rda("/home/jovyan/suspect/tests/test_data/siemens/SVS_30.rda")

## LCModel

LCModel requires several different files in order to process a spectrum. The actual MRS data is stored in the time domain in a .RAW file, with an optional .H20 file containing the water reference data (these are actually the same format but the extension helps in distinguishing the files). A .CONTROL file is what the LCModel program actually receives as input, it lists the input and output files, and any parameters which have non-default values. Finally the metabolite basis set used for fitting the data is contained in a .BASIS file. The .BASIS file is normally provided with LCModel and unlike the other files does not need to be changed with each new dataset.

Suspect can generate all the files necessary to process data with LCModel using the `write_all_files()` function from the `suspect.io.lcmodel` module. This function takes a path to which to save the .RAW file (any other files will be saved in the same directory), the `MRSData` object to be written, and an optional `params` dictionary to customise any of the values in the .CONTROL file. This dictionary can contain any of the parameter names from the LCModel manual. Let's take a look at an example of how to save our data for LCModel to process.

In [3]:
# create a parameters dictionary to set the basis set to use
params = {
    "FILBAS": "/path/to/lcmodel/basis.BASIS"
}
suspect.io.lcmodel.write_all_files("lcmodel_data/example.RAW", data, params=params)

We can use some IPython magic to show the files that were created:

In [4]:
!ls lcmodel_data/

example.RAW  example_sl0.CONTROL


and to look at the contents of the .CONTROL file:

In [5]:
!cat lcmodel_data/example_sl0.CONTROL

 $LCMODL
 OWNER = ''
 KEY = 123456789
 DELTAT = 0.000833
 HZPPPM = 123.234655
 NUNFIL = 1024
 NDROWS = 1
 NDCOLS = 1
 FILBAS = '/path/to/lcmodel/basis.BASIS'
 NDSLIC = 1
 FILPS = 'lcmodel_data/example.PS'
 ICOLST = 1
 FILRAW = 'lcmodel_data/example.RAW'
 IROWEN = 1
 ICOLEN = 1
 DOWS = F
 IROWST = 1
 DOECC = F
 $END


The .CONTROL file contains all the parameters necessary for LCModel to process the file, including the path we specified to the correct basis set. Once the .RAW and .CONTROL files are generated, it only remains to run the LCModel program on the command line, passing in the .CONTROL file like this:

    lcmodel < lcmodel_data/example_sl0.CONTROL

If you are running your Suspect code on the same computer as your LCModel installation, so that lcmodel is in your path, it is trivial to run it from within a Jupyter notebook using IPython magic, or from a script using the `subprocess` module. However if you are running Suspect on a different computer, for example using the OpenMRSLab Docker container, then you will have to transfer the LCModel files to the LCModel computer for processing. This can be inconvenient because the paths in the generated .CONTROL file can become out of date. In our lab we have solved this problem by using a shared network drive which is mounted by both machines, then the paths remain consistent and the `lcmodel` program can be launched remotely from the Docker container with a simple ssh command. Another alternative is to use the `suspect.io.lcmodel.save_raw()` function instead to save only the .RAW file which can then be loaded by LCMGUI and the other parameters configured there. 

By default the control file is set to only generate the standard 1 page .PS output. The other output files can be generated by setting the appropriate options in the params dictionary. For example, to generate the .CSV file, set the "LCSV" parameter to True. In this case, Suspect will automatically generate the path for the .CSV in the same folder as the .RAW and .CONTROL files, to set a custom location to save the .CSV, instead set the parameter "FILCSV" in params.

In [6]:
# create a parameters dictionary to set the basis set to use
params = {
    "FILBAS": "/path/to/lcmodel/basis.BASIS",
    "LCSV": True
}
suspect.io.lcmodel.write_all_files("lcmodel_data/example.RAW", data, params=params)

In [7]:
!cat lcmodel_data/example_sl0.CONTROL

 $LCMODL
 OWNER = ''
 KEY = 123456789
 DELTAT = 0.000833
 HZPPPM = 123.234655
 NUNFIL = 1024
 NDROWS = 1
 NDCOLS = 1
 FILBAS = '/path/to/lcmodel/basis.BASIS'
 NDSLIC = 1
 FILPS = 'lcmodel_data/example.PS'
 ICOLST = 1
 FILRAW = 'lcmodel_data/example.RAW'
 IROWEN = 1
 ICOLEN = 1
 LCSV = 11
 DOWS = F
 IROWST = 1
 FILCSV = 'lcmodel_data/example.CSV'
 DOECC = F
 $END
